<a href="https://colab.research.google.com/github/itsthemoon/chatBot/blob/main/chatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLTK UTILS

In [1]:
pip install nltk

In [2]:
import nltk 
import numpy as np
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)


In [4]:
def stem(word):
  return stemmer.stem(word.lower())

In [5]:
def bag_of_words(tokenized_sentence, all_words):
  """
  Example:
  sentence: ["hello", "how", "are", "you"]
  words: ["hi", "hello", "I", "you", "bye", "thank", "cool"]
  bag =  [  0,      1,     0,    1,      0,      0,      0]
  """
  tokenized_sentence = [stem(w) for w in tokenized_sentence]

  bag = np.zeros(len(all_words), dtype=np.float32)
  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0
  
  return bag 
  

# Train


In [7]:
import json
import numpy as np
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
with open('intents.json', 'r') as f:
  intents = json.load(f)



In [8]:
all_words = []
tags = []
xy = []
for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w, tag))

ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
  bag = bag_of_words(pattern_sentence, all_words)
  X_train.append(bag)

  label = tags.index(tag)
  y_train.append(label) #CrossEntropyLoss 

X_train = np.array(X_train)
y_train = np.array(y_train)


In [9]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(X_train)
    self.x_data = X_train
    self.y_data = y_train

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.n_samples




In [14]:
#hyperparams
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size = batch_size, 
                          shuffle=True, num_workers=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #use gpu
print(device)
model = Net(input_size, hidden_size, output_size).to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [18]:
for epoch in range(epochs):
  for (words, labels) in train_loader:
    words = words.to(device)
    labels = labels.to(device)

    #forward
    output = model(words)
    loss = criterion(output, labels)

    #backwards and optimizer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch + 1) % 100 == 0:
    print(f'epoch {epoch+1}/{epochs}, loss={loss.item():.4f}')

print(f'final loss, loss={loss.item():.4f}')

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

epoch 100/1000, loss=1.2041
epoch 200/1000, loss=0.7287
epoch 300/1000, loss=0.0225
epoch 400/1000, loss=0.0901
epoch 500/1000, loss=0.0175
epoch 600/1000, loss=0.0017
epoch 700/1000, loss=0.0017
epoch 800/1000, loss=0.0037
epoch 900/1000, loss=0.0003
epoch 1000/1000, loss=0.0014
final loss, loss=0.0014
training complete. file saved to data.pth


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Model

In [13]:
import torch
import torch.nn as nn 

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()
  
  def forward(self, x):
    x = self.l1(x)
    x = self.relu(x)
    x = self.l2(x)
    x = self.relu(x)
    x = self.l3(x)
    #no activation and no softmax 
    return x     
